In [1]:
import cv2
import numpy as np
from utils import *
from segmentation import breakLines, breakWords, extractTemplate, templateMatch, getBaseLine, addEmptySpaceSep, matchFirstCharacter

In [ ]:
def datasetToCsv(img_dir, text_dir, image_files, destination):
    rows = []

    column_names = ["feature" + str(i) for i in range(25*25)]
    column_names.append("label")
    count = 0
    for image in image_files:
        if limit > 0 and count == limit:
            break
        if count % 200 == 0:
            print("currently at image: " + str(count))
        img_path = img_dir + "/" + image
        text_path = text_dir + "/" + image.split('.')[0] + ".txt"

        img = cv2.imread(img_path)
        img_words = imageToWords(img)
        with open(text_path) as f:
            text = f.readlines()[0]
            text_words = text.split()
            index = 0
            for word in text_words:
                if index >= len(img_words):
                    break
                chars = segmenteCharacters(img_words[index])
                if len(chars) == len(word):
                    chars.reverse()
                    for i in range(len(word)):
                        letter = extractTemplate(chars[i])
                        if letter.shape[0] < 25 and letter.shape[1] < 25:
                            mask = np.zeros((25, 25))

                            vertical_start = int((25 - letter.shape[0]) / 2)
                            vertical_end = vertical_start + letter.shape[0]
                            horizontal_start = int((25 - letter.shape[1]) / 2)
                            horizontal_end = horizontal_start + letter.shape[1]

                            mask[vertical_start:vertical_end,
                                 horizontal_start:horizontal_end] = letter
                            mask = mask.reshape(1, mask.shape[0]*mask.shape[1])
                            row = [mask[:, i][0] for i in range(mask.shape[1])]
                            row.append(word[i])
                            rows.append(row)
                            LETTER_COUNTS[word[i]] += 1
                index += 1
        count += 1

    df = pd.DataFrame(data=rows, columns=column_names)
    df.to_csv(destination + "/dataset.csv", index=False)